# Scraping Weather Data With Python
Hello everyone! Welcome to another one of my projects. This time I'll be looking into how to scrape data off the internet automatically using Python, specifically BeautifulSoup and urllib2. Although not the most complicated project, as I don't have to manually sift the data but rather just centralise it all, it should still serve as useful for the community. 

This Jupyter notebook is purely to show the proof-of-concepts for my processes such as requesting, downloading and sorting the data. The full collection of the data will be done by an external python file which has to be executed in the terminal.


## Overview Of The Project
The project's main aim is to collect weather data from all over Western Australia and centralise it into this repository. In theory this process could be extended to Australia-wide, but I don't want that much data. I'll illustrate how one could do that later in the project. That way others can use it in their own projects without needing to repeat the process I'm about to undertake.

The data I've collected is simply the options specified by the Bureau of Meteorology on their [Climate Data database](http://www.bom.gov.au/climate/data/). As such, I'll be collecting information on: Rainfall; Temperature (Max & Min); and Solar Exposure.

My general thought process in storing the data will be through the use of a 3D array (known as a [Panel](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Panel.html) in Pandas) with:
 - Each 'slice' (which forms a 2D matrix) repesenting a station.
 - Each row representing a date.
 - Each column representing a piece of data.

I've attempted to represent a slice of the data for station $n$ below.

$$
\begin{bmatrix}
Date & Rainfall(mm) & ...\\
01/01/2015 & 12.3 & ...\\ 
02/01/2015 & 6.1 & ...\\ 
... & ... & ...
\end{bmatrix}
$$



## Usage of The Data
For anyone reading this notebook who might be interested in using my data that I've collected, I give full permission without any attribution needed. Go forth and solve the world's problems using data!

All data has been scraped from [The Bureau of Meteorology](http://www.bom.gov.au). I claim rights to none of the data. As such I recommend reading into their data policies before using it for commercial use (but I'm sure personal use will be fine).

---
With all of that out of the way, let's begin!


In [1]:
# Import all of the goodies that I'll be using.

# HTML Scraping Packages
import urllib2
from bs4 import BeautifulSoup

# File Manipulation Packages
import zipfile
import glob

# Data Manipulation Packages
import pandas

In [2]:
# Next, define some global variables that I'll be using.
BOM_HOME = r'http://www.bom.gov.au'

---
# Part 1 - Finding The Data
The first step in this project will be finding the data to collect. As already mentioned, you can find most of the data online at the [Climate Data database](http://www.bom.gov.au/climate/data/). However, collecting data this way is really tedious as you would have to manually fill in forms. Although this is possible with Python packages such as [Selenum](http://selenium-python.readthedocs.io/) there is definately an easier way.

---

## 1.1. Tinkering With The Portal

The first thing I did was to gain some familarity with how the portal worked. So I searched up for a weather station, found it's station number and went to that data page. If you'd like to follow along, I used station 9021.

Once I went to the new page, I noticed that the URL was structured like so:

> http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear=&p_c=&p_stn_num=009021

So already you can see the URL contains a query string. For the most part I can't really tell what each parameter does, but I did notice that my station number was in the string:

> p_stn_num=009021

So naturally I played around with this parameter. Of course this enabled me to move to new weather station! Let me illustrate that below.

In [3]:
# The name of the weather station for a given station number
def getStationName(station_num):
    page = urllib2.urlopen(
        r'http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear=&p_c=&p_stn_num=' 
        + str(station_num).zfill(6)
    )
    soup = BeautifulSoup(page, 'html.parser')
    return soup.h2.string

In [4]:
print "Station 9021:", getStationName(9021)
print "Station 9022:", getStationName(9022)

Station 9021: Perth Airport 
Station 9022: Guildford Post Office 


If you explore this further however you'll notice that not all stations exist. Uncomment the code below if you want to see this in action.

In [5]:
# print getStationName(9)

So we need to find the numbers of all of the stations in WA. Fortunately that's really easy!

---

## 1.2. Finding The Weather Stations That Exist
Although you could simply test all 6-digit numbers and capture any that exist, it is far easier to just do a simply google search. The BOM actually all you to query for a list of the stations [here](http://www.bom.gov.au/climate/data/stations/). At this point I also noticed a lot more data that I could collect if I desired.

I also noticed a big problem. You can simply request the data from BOM instead of collecting it manually! For the sake of practice I will however just collect the data manually. 

[This text file](http://www.bom.gov.au/climate/data/lists_by_element/alphaWA_136.txt) lists all of the weather stations that collect rainfall data in WA. So that means that I can just use this list to collect my data!

The text file is structured for humans to read, not a computer to. At this point I used excel to convert the text file into a CSV. After a little bit of cleaning up of the headings and footers, I imported the CSV into a Pandas dataframe so we can access the information easily.

In [6]:
df_allstations = pandas.read_csv('rainfall_stations_wa.csv')
df_allstations.head()


,Site,Name,Lat,Lon,Start,End,Years,%,AWS
0,7118,ABBOTTS,-26.4000,118.4000,Sep 1898,Nov 1913,6.8,45,N
1,10258,ABERVON,-30.7833,117.9833,May 1968,Aug 1973,5.2,97,N
2,4000,ABYDOS,-21.4167,118.9333,Jul 1917,Dec 1974,40.0,70,N
3,4045,ABYDOS WOODSTOCK,-21.6200,118.9550,Apr 1901,Sep 1997,65.2,67,N
4,9971,ACTON PARK,-33.7845,115.4072,Nov 2000,Nov 2017,17.0,98,N


Most of the titles used in the data are self-explainatory. However, let me explain the last two columns:
 1. **Percentage (%)**: After tinkering around with the data, I think it is a measure of the completeness of the data collected. The higher the percentage, the lower the number of missing data points. Obviously 100% indicates that there aren't any missing data points at all during its time of operation.
 2. **AWS**: This column signifies that the weather station is an *Automatic Weather Station*. The BOM state that their readings are more consistent and accurate, which would be an interesting observation to back up with data.

---

# Part 2 - Collecting The Data
Now we know what data exists, we can go about attempting to collect it. This section will mainly focus on the downloading, extracting and manipulating the data into a format where it can be easily restructured.

---

## 2.1. Downloading Historical Data
Now that we know which stations exist, we want to explore if it is possible to automate the downloading process.

On each weather station page, there is a button to download all historical data on the page. Thus we want to target this URL using Beautiful Soup. By inspecting the source code on each page, it is contained within the "downloads" class as the second item in the list. Let's try to target that.

In [7]:
def downloadDataLink(station_num):
    page = urllib2.urlopen(
        r'http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear=&p_c=&p_stn_num=' 
        + str(station_num).zfill(6)
    )
    soup = BeautifulSoup(page, 'html.parser')
    download_link_extension = soup.find(
        'a', 
        {'title': "Data file for daily rainfall data for all years"}
    )['href']
    return str(BOM_HOME + download_link_extension)

I know that's a little complicated looking. Let me break it down for you:
 1. Read the station data page. This is simply where you get to if you manaully input in the station number into the portal. The zfill is to ensure that the number is formatted correctly.
 2. Get the HTML code from Beautiful Soup.
 3. Extract the download link. You can find it by targeting the correct title.
 4. The link is only appended to the base BOM link. So return the full link by concatinating the two strings together.
 
You can see how this output looks for different stations below.

In [8]:
print downloadDataLink(9021)
print downloadDataLink(9022)

http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_display_type=dailyZippedDataFile&p_stn_num=009021&p_c=-16487464&p_nccObsCode=136&p_startYear=2017
http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_display_type=dailyZippedDataFile&p_stn_num=009022&p_c=-16491073&p_nccObsCode=136&p_startYear=1954


Downloading the data from here is actually really simple. If you open the URL using urllib2 you'll receive the entire source code, which you can simply save by writing that code into a zipped file.

I've done this below to show how this could be done.

In [9]:
data = urllib2.urlopen(downloadDataLink(9021))
with open('station_9021.zip', 'wb') as zipper:
    zipper.write(data.read())

From this download we get a zipped file with two text files inside: the first is the actual data while the second is information about the weather station. In theory we could throw out the second file, but for now I'm keeping it.

---
## 2.2 Extracting and Manipulating Data
Now that we have the data, we want to be able to import it! That way we can begin to format it together.

I'll start with extracting the zipped file, and viewing the first few lines of the CSV.

In [10]:
with zipfile.ZipFile("station_9021.zip", "r") as zip_ref:
    zip_ref.extractall("station_9021")
    
df_station9021 = pandas.read_csv(glob.glob("station_9021/*.csv")[0])
print "(Rows, Columns) =", df_station9021.shape
df_station9021.head()

(Rows, Columns) = (27009, 8)


,Product code,Bureau of Meteorology station number,Year,Month,Day,Rainfall amount (millimetres),Period over which rainfall was measured (days),Quality
0,IDCJAC0009,9021,1944,1,1,NaN,NaN,NaN
1,IDCJAC0009,9021,1944,1,2,NaN,NaN,NaN
2,IDCJAC0009,9021,1944,1,3,NaN,NaN,NaN
3,IDCJAC0009,9021,1944,1,4,NaN,NaN,NaN
4,IDCJAC0009,9021,1944,1,5,NaN,NaN,NaN


Upon first inspection, I can already think of ways to simplify the data. Firstly I will be dropping the first column as it is effectively useless to me. The second column isn't actually much use, as we already know what the station number is. I will be renaming the sixth column to a shorter name as well as aim to combine columns 3-5 into a single date column. 

In [11]:
# Import in the dataframe.
df_station9021 = pandas.read_csv(glob.glob("station_9021/*.csv")[0])

# Rename the columns.
df_station9021.rename(columns={
    'Bureau of Meteorology station number': "Station Number",
    'Rainfall amount (millimetres)': 'Rainfall',
}, inplace=True)

# Combine date into a single column.
# (I know this code sucks, I couldn't make it work any other way).
df_station9021['Year'] = df_station9021['Year'].map(str)
df_station9021['Month'] = df_station9021['Month'].map(lambda x: str(x).zfill(2))
df_station9021['Day'] = df_station9021['Day'].map(lambda x: str(x).zfill(2))

df_station9021.insert(
    2, 'Date', 
    
    df_station9021['Year'] + '-' + 
    df_station9021['Month'] + '-' + 
    df_station9021['Day']
)

df_station9021.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

# Next, drop the first column.
df_station9021.drop(["Product code", "Station Number"], axis=1, inplace=True)

# Visualise the newly formatted data.
print "(Rows, Columns) =", df_station9021.shape
df_station9021.head()

(Rows, Columns) = (27009, 4)


,Date,Rainfall,Period over which rainfall was measured (days),Quality
0,1944-01-01,NaN,NaN,NaN
1,1944-01-02,NaN,NaN,NaN
2,1944-01-03,NaN,NaN,NaN
3,1944-01-04,NaN,NaN,NaN
4,1944-01-05,NaN,NaN,NaN


It all works! Now the data is in an easy to access format and is only 5 columns instead of the original 8. Of course the above process can be repeated for each station in the data, but this is only a proof of concept. The full data processing will be done via the executable python script.

---
## Aside: How Clean Is The Data?
I was thinking to myself that the data I'm showing doesn't look particularly complete. So I'll compute how many missing entries are in this particular station and see how that compares to the number given in the Station database.

In [12]:
missing_values = df_station9021.isnull().sum()
print missing_values
print (missing_values / df_station9021.shape[0]) * 100

Date                                                  0
Rainfall                                            121
Period over which rainfall was measured (days)    17935
Quality                                             121
dtype: int64
Date                                               0.000000
Rainfall                                           0.447999
Period over which rainfall was measured (days)    66.403791
Quality                                            0.447999
dtype: float64


So, actually, the data isn't missing much information at all. Why are there so many missing entries at the start then?

I managed to figure it out. Although the weather station didn't open until May, the CSV still has "empty" data for the first 4 months of the year. So to get a better picture of how complete the data is, we should throw all data before the first entry and all data after the last entry.

In [13]:
data_start = df_station9021['Rainfall'].first_valid_index()
data_finish = df_station9021['Rainfall'].last_valid_index()
print data_start, data_finish

df_station9021 = df_station9021[data_start:data_finish]
print df_station9021.head(3), '\n\n\n\n', df_station9021.tail(3)

121 27008
           Date  Rainfall  Period over which rainfall was measured (days)  \
121  1944-05-01       0.0                                             NaN   
122  1944-05-02       0.0                                             NaN   
123  1944-05-03       0.0                                             NaN   

    Quality  
121       Y  
122       Y  
123       Y   



             Date  Rainfall  Period over which rainfall was measured (days)  \
27005  2017-12-08       0.0                                             1.0   
27006  2017-12-09       0.0                                             1.0   
27007  2017-12-10       0.0                                             1.0   

      Quality  
27005       N  
27006       N  
27007       N  


Now, if I were to repeat the same analysis I should get near perfect data!

In [14]:
missing_values = df_station9021.isnull().sum()
print missing_values
print (missing_values / df_station9021.shape[0]) * 100

Date                                                  0
Rainfall                                              0
Period over which rainfall was measured (days)    17814
Quality                                               0
dtype: int64
Date                                               0.000000
Rainfall                                           0.000000
Period over which rainfall was measured (days)    66.255068
Quality                                            0.000000
dtype: float64


Yes! There are no missing data for the rainfall at the Perth airport, as I infered (since it is an important weather station).

---

# Part 3 - Centralising The Data
Now that we know it is possible to collect, simplify and clean the data we can focus on trying to centralise it into a single data structure. My thoughts immediately went to using Pandas' [Panels](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Panel.html) but I've rethought my approach. Instead I'll be using the [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/advanced.html) object.

---

## 3.1. Designing The Data Structure
Now that we know how we will want each dataframe to look, we can now focus on the structure of the data as a bigger picture. Since we will be using hierarchial indexing the dataframe will look like so:

```
+---                    ---+
| 9021  1900-01-01  ...    |
|       1900-01-02  ...    |
|       ...                |
| 9021  1900-01-01  ...    |
|       1900-01-02  ...    |
|       ...                |
| ...   ...                |
+---                    ---+
```

So the outermost index is the station number and the innermost index is the date. I've decided upon this structure, instead of the other way, since it is more likely that we will be focusing on a single station rather than a single date.

Let's explore this concept with two station's worth of data to see how easy it is to do.

In [15]:
def createDataframeForStation(station_num):
    """Given the station number, I'll fetch the data for it."""
    download_name = 'station_%s' % station_num
    
    # Fetch & download data
    link = downloadDataLink(station_num)
    data = urllib2.urlopen(link)
    with open(download_name+'.zip', 'wb') as zipper:
        zipper.write(data.read())
    
    # Extract it 
    with zipfile.ZipFile(download_name+".zip", "r") as zip_ref:
        zip_ref.extractall(download_name)
    df = pandas.read_csv(glob.glob(download_name+"/*.csv")[0])
    
    # Rename the columns
    df.rename(columns={
        'Bureau of Meteorology station number': "Station Number",
        'Rainfall amount (millimetres)': 'Rainfall',
    }, inplace=True)
    
    # Combine date into a single column.
    df['Year'] = df['Year'].map(str)
    df['Month'] = df['Month'].map(lambda x: str(x).zfill(2))
    df['Day'] = df['Day'].map(lambda x: str(x).zfill(2))
    df.insert(
        2, 'Date', 
        df['Year'] + '-' + 
        df['Month'] + '-' + 
        df['Day']
    )
    df.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

    # Next, drop the first and second columns.
    df.drop(["Product code", "Station Number"], axis=1, inplace=True)
    
    # Set the date as the index column.
    df.set_index('Date', inplace=True)
    
    # Finally return the result.
    return df

I should see if that worked first.

In [16]:
createDataframeForStation(9021).tail(5)

,Rainfall,Period over which rainfall was measured (days),Quality
Date,,,
2017-12-07,0.0,1.0,N
2017-12-08,0.0,1.0,N
2017-12-09,0.0,1.0,N
2017-12-10,0.0,1.0,N
2017-12-11,0.0,1.0,N


Which it did! So let's collect data for station 9021, 9022 and 9023 so we can start playing around with it.

In [17]:
# Get the station data.
df9021 = createDataframeForStation(9021)
df9022 = createDataframeForStation(9022)
df9023 = createDataframeForStation(9023)

# Combine it into a single dataframe.
df_main = pandas.concat(
    [df9021, df9022, df9023], 
    keys=['9021', '9022', '9023'], 
    names=['Station Number', 'Date']
)

Did it work? Well we can see if we can access a single station, a collection of stations or all data by a specific date.

In [18]:
df_main.loc['9022'].head()

,Rainfall,Period over which rainfall was measured (days),Quality
Date,,,
1877-01-01,0.0,NaN,Y
1877-01-02,0.0,NaN,Y
1877-01-03,0.0,NaN,Y
1877-01-04,0.0,NaN,Y
1877-01-05,0.0,NaN,Y


In [19]:
df_main.loc[['9021', '9022']]

Rainfall  \
Station Number Date                   
9021           1944-01-01       NaN   
               1944-01-02       NaN   
               1944-01-03       NaN   
               1944-01-04       NaN   
               1944-01-05       NaN   
               1944-01-06       NaN   
               1944-01-07       NaN   
               1944-01-08       NaN   
               1944-01-09       NaN   
               1944-01-10       NaN   
               1944-01-11       NaN   
               1944-01-12       NaN   
               1944-01-13       NaN   
               1944-01-14       NaN   
               1944-01-15       NaN   
               1944-01-16       NaN   
               1944-01-17       NaN   
               1944-01-18       NaN   
               1944-01-19       NaN   
               1944-01-20       NaN   
               1944-01-21       NaN   
               1944-01-22       NaN   
               1944-01-23       NaN   
               1944-01-24       NaN   
               1944-01-25       NaN   
               1944-01-26       NaN   
               1944-01-27       NaN   
               1944-01-28       NaN   
               1944-01-29       NaN   
               1944-01-30       NaN   
...                             ...   
9022           1954-06-01       0.8   
               1954-06-02       0.0   
               1954-06-03       0.0   
               1954-06-04       0.5   
               1954-06-05       0.0   
               1954-06-06       0.0   
               1954-06-07       0.0   
               1954-06-08       0.0   
               1954-06-09       0.0   
               1954-06-10       0.0   
               1954-06-11       0.0   
               1954-06-12       0.0   
               1954-06-13       0.0   
               1954-06-14       1.3   
               1954-06-15       2.8   
               1954-06-16       0.0   
               1954-06-17      13.0   
               1954-06-18       1.0   
               1954-06-19      20.1   
               1954-06-20      27.7   
               1954-06-21       0.8   
               1954-06-22       0.3   
               1954-06-23       0.0   
               1954-06-24       1.0   
               1954-06-25       4.3   
               1954-06-26      10.4   
               1954-06-27      22.6   
               1954-06-28      18.0   
               1954-06-29       0.3   
               1954-06-30       0.0   

                           Period over which rainfall was measured (days)  \
Station Number Date                                                         
9021           1944-01-01                                             NaN   
               1944-01-02                                             NaN   
               1944-01-03                                             NaN   
               1944-01-04                                             NaN   
               1944-01-05                                             NaN   
               1944-01-06                                             NaN   
               1944-01-07                                             NaN   
               1944-01-08                                             NaN   
               1944-01-09                                             NaN   
               1944-01-10                                             NaN   
               1944-01-11                                             NaN   
               1944-01-12                                             NaN   
               1944-01-13                                             NaN   
               1944-01-14                                             NaN   
               1944-01-15                                             NaN   
               1944-01-16                                             NaN   
               1944-01-17                                             NaN   
               1944-01-18                                             NaN   
               1944-01-19    

In [20]:
df_main.xs('1950-01-01', level='Date')

,Rainfall,Period over which rainfall was measured (days),Quality
Station Number,,,
9021,0.0,NaN,Y
9022,0.0,NaN,Y
9023,0.0,NaN,Y


In [21]:
df_main.xs('1900-01-01', level='Date')

,Rainfall,Period over which rainfall was measured (days),Quality
Station Number,,,
9022,NaN,NaN,NaN
9023,NaN,NaN,NaN


It all seems to work exactly as expected!

## 3.2 Exporting The Result
Now that we have got a good protoype of the data structure, we want ot confirm that we can actually export it to a CSV file for others to use. Fortunately for me, pandas has a [to_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html) function that I'll be able to use.

Let's test this function on the dataframe we just generated of stations 9021, 9022 and 9023.

In [23]:
df_main.to_csv('example.csv')

Well that was stupidly-easy. In fact that was too easy. Did the data actually export as I wanted?

In [26]:
with open('example.csv', 'r') as f:
    for _ in range(5): print f.readline()

Station Number,Date,Rainfall,Period over which rainfall was measured (days),Quality

9021,1944-01-01,,,

9021,1944-01-02,,,

9021,1944-01-03,,,

9021,1944-01-04,,,



Well it sure looks like it. What happens if I re-export it back into Pandas?

In [31]:
importdf = pandas.read_csv("example.csv")
importdf.tail(5)

,Station Number,Date,Rainfall,Period over which rainfall was measured (days),Quality
104961,9023,2017-12-07,0.0,1.0,N
104962,9023,2017-12-08,0.0,1.0,N
104963,9023,2017-12-09,0.0,1.0,N
104964,9023,2017-12-10,0.0,1.0,N
104965,9023,2017-12-11,0.0,1.0,N


And it all still works as I expected! The import is even better if I use the *index_col* parameter in the read_csv function.